<a href="https://colab.research.google.com/github/kevin36524/AdapterTesting/blob/master/ChatGPTFineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU \
    datasets==2.14.4 \
    langchain==0.0.274 \
    pinecone-client==2.2.2 \
    openai==0.27.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [6]:
from datasets import load_dataset

data = load_dataset(
    'json',
    data_files= "finalTrainingSet.jsonl",
    split="train"
)
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages', 'functions'],
    num_rows: 52
})

In [8]:
print(data)
data[0]

Dataset({
    features: ['messages', 'functions'],
    num_rows: 52
})


{'messages': [{'role': 'user',
   'content': 'Extract calendar events from EMAIL_BODY for the time zone "UTC" as of "2023-03-09T06:00:49Z". \n    - Only Extract events if EMAIL_BODY is related the following topics : "appointment", "meeting", "class", "school", "doctor", "clinic", "hospital", "workshop", "conference", "seminar", "session", "car service", "interview", "reservation", "booking". \n    - Do not extract events if EMAIL_BODY is related to the following keywords or topics : "finance", "banks", "sale", "offer", "discount", "promotions", "buy now", "limited time". "click baits", "spams"\n    - If event time zone is not specified, default to "UTC"\n    EMAIL_BODY: \n Add Paperless Post to your address book to receive all cards and invitations from Sasha and James Rochabrun on time.\\nThe delivery of this email may have been delayed due to spam blocking settings.\\n\\nFor: Kevin Patel\\n\\nVIEW THE CARD\\n\\n\\n\\nISABELAS THIRD BIRTHDAY\\nSaturday, April 1st from 2:30 PM to 5:30 

In [9]:
data.to_json("conversations.jsonl")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

187975

In [11]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]

res = openai.File.create(
    file=open("finalTrainingSet.jsonl", "r"),
    purpose='fine-tune'
)
res

<File file id=file-8aeFW4ZVj6mKGgfUvQvbDNO6 at 0x78ffb45fd6c0> JSON: {
  "object": "file",
  "id": "file-8aeFW4ZVj6mKGgfUvQvbDNO6",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 186052,
  "created_at": 1699058104,
  "status": "processed",
  "status_details": null
}

In [12]:
file_id = res["id"]
file_id

'file-8aeFW4ZVj6mKGgfUvQvbDNO6'

In [13]:
res = openai.FineTuningJob.create(training_file=file_id, model="gpt-3.5-turbo")
res


<FineTuningJob fine_tuning.job id=ftjob-9asVl9yDbkXL0sAkgFKWAAML at 0x78ffa079eb10> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-9asVl9yDbkXL0sAkgFKWAAML",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699058134,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-zjZK5cg4WULtZnztRVM5Bbm9",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-8aeFW4ZVj6mKGgfUvQvbDNO6",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [14]:
job_id = res["id"]
job_id

'ftjob-9asVl9yDbkXL0sAkgFKWAAML'

In [29]:
openai.FineTuningJob.retrieve(job_id)


<FineTuningJob fine_tuning.job id=ftjob-9asVl9yDbkXL0sAkgFKWAAML at 0x78ffa0684ef0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-9asVl9yDbkXL0sAkgFKWAAML",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1699058134,
  "finished_at": 1699058642,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:yahoo-comms::8Gz3cWT5",
  "organization_id": "org-zjZK5cg4WULtZnztRVM5Bbm9",
  "result_files": [
    "file-2oCKWjOVzKFeWFIMjEybqPaF"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-8aeFW4ZVj6mKGgfUvQvbDNO6",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 130161,
  "error": null
}